<a href="https://colab.research.google.com/github/VasquezSRE/machinelearning-gtc/blob/main/extraction/Feature%20Selection/2_ANN_FwrdSln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sys; sys.path.append('./Utils')
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
import pandas as pd
from sklearn.neural_network import MLPClassifier
from numpy import random
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from numpy import matlib
import qgrid
from sklearn.model_selection import train_test_split
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix
import sys; sys.path.append('./')
from utils import Utils
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import time
import qgrid

In [ ]:
ut = Utils()

In [ ]:
X, Y =ut.get_matrixes()

In [ ]:
X.shape

(4965, 23)

In [ ]:
def select_features(modelo, n_features, fwd, fltg):
    sfs = SFS(modelo, 
           k_features=n_features, 
           forward=fwd,
           floating=fltg,
           verbose=2,
           scoring='accuracy',
           cv=0,
           n_jobs=-1,
           pre_dispatch=10)
    
    return sfs

In [ ]:
def trainSFS(n_features,clf):
    
    forward = True
    floating = False
   
        
        
    tiempo_i = time.time()
    
    #Implemetamos la metodología de validación 

    Folds = 4
    Errores = np.ones(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X, Y, random_state=0, stratify=Y, shuffle=True)
    for train, test in skf.split(X_train_d,y_train_d):
        

        X_train, X_test = X[train], X[train]
        y_train, y_test = Y[train], Y[train]  
        #Aquí se entrena y se valida el modelo haciendo selección de características con diferentes estrategias

        #Complete el código llamando el método select_features con los parámetros correspondientes para responder el
        #Ejercicio 3.1
        sf = select_features(clf, n_features, forward, floating)

        #Complete el código para entrenar el modelo con las características seleccionadas. Tenga en cuenta
        #la metodología de validación aplicada para que pase las muestras de entrenamiento correctamente.
        print("iteracion: " + str(j))
        print("feature: " + str(n_features))
        sf = sf.fit(X_train, y_train)

        Errores[j] = 1-sf.k_score_
        j+=1
    return  str(np.mean(Errores)), str(np.std(Errores)), str(sf.k_score_*100), str(time.time()-tiempo_i),sf.k_feature_idx_

In [ ]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [ ]:
def train(clf):
    #Para calcular el costo computacional
    tiempo_i = time.time()

    #Implemetamos la metodología de validación

    j = 0
    Folds = 4
    Errores = np.ones(Folds)
    skf = StratifiedKFold(n_splits=Folds)

    for train, test in skf.split(X,Y):

        Xtrain, Ytrain, Xtest, Ytest = ut.get_training_test(X=X, Y=Y, train=train, test=test)

        #Aquí se entran y se valida el modelo sin hacer selección de características

        ######

        # Entrenamiento el modelo.
        model = clf.fit(Xtrain,Ytrain)

        # Validación del modelo
        ypred = model.predict(Xtest)

        #######

        Errores[j] = classification_error(ypred, Ytest)
        j+=1
    
    m_errores = np.mean(Errores)
    ic_errores = np.std(Errores)
    eficiencia = (1 - np.mean(Errores))*100
    tEjec = time.time()-tiempo_i
                        
    print('finalizado clf sin selección',str(m_errores), str(ic_errores), str(eficiencia), str(tEjec))
    return  m_errores, str(np.std(Errores)), str((1 - np.mean(Errores))*100), str(time.time()-tiempo_i)

In [ ]:
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['MLP sin Selección','MLP + SFS','MLP + SFS']),
    '# de características seleccionadas' : pd.Series([23,15,20]),
   })
ic_ = []
tEjec_ = []
errorVal_ = []
eficiencia_ = []
caracteristicas=[]
model = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes = (20, 20), activation = 'relu', max_iter=500), n_jobs=-1)
for e, i in zip(df_types['Tecnica'], df_types['# de características seleccionadas']):
    
    if(e == "MLP sin Selección"):
        errorVal, ic, eficiencia, tEjec = train(model)
        errorVal_.append(errorVal)
        ic_.append(ic)
        tEjec_.append(tEjec+" s")
        eficiencia_.append(eficiencia)
    else:
        print('entrenando: ',e,i)
        errorVal, ic, eficiencia, tEjec,car = trainSFS(i, model)
        errorVal_.append(errorVal)
        ic_.append(ic)
        tEjec_.append(tEjec+" s")
        eficiencia_.append(eficiencia)
        caracteristicas.append(car)



finalizado clf sin selección 0.06888194030838463 0.027041804478908832 93.11180596916154 62.42895030975342
entrenando:  MLP + SFS 15
iteracion: 0
feature: 15


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:  1.1min finished

[2021-01-20 08:07:59] Features: 1/15 -- score: 0.7782951289398281[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:  2.8min finished

[2021-01-20 08:10:46] Features: 2/15 -- score: 0.8635386819484241[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  3.2min finished

[2021-01-20 08:13:59] Features: 3/15 -- score: 0.8939828080229226[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.7min finished

[2021-01-20 08:17:38] Features: 4/15 -- score: 0.9129656160458453[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:  4.0min finished



iteracion: 1
feature: 15


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:  1.4min finished

[2021-01-20 08:51:26] Features: 1/15 -- score: 0.7707736389684814[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:  3.1min finished

[2021-01-20 08:54:29] Features: 2/15 -- score: 0.8402578796561605[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  3.3min finished

[2021-01-20 08:57:50] Features: 3/15 -- score: 0.877865329512894[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.9min finished

[2021-01-20 09:01:47] Features: 4/15 -- score: 0.8975644699140402[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:  3.8min finished

[2021-01-20 09:05:35] Features: 5/15 -- score: 0.9226361031518625[Parallel(n_

iteracion: 2
feature: 15


...

[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:  1.2min finished

[2021-01-20 11:49:36] Features: 1/20 -- score: 0.7725644699140402[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:  3.2min finished

[2021-01-20 11:52:46] Features: 2/20 -- score: 0.8406160458452722[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  3.8min finished

[2021-01-20 11:56:31] Features: 3/20 -- score: 0.8889684813753582[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  4.0min finished

[2021-01-20 12:00:31] Features: 4/20 -- score: 0.9029369627507163[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:  4.1min finished

[2021-01-20 12:04:34] Features: 5/20 -- score: 0.920487106017192[Parallel(n_

iteracion: 2
feature: 20


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:  1.2min finished

[2021-01-20 12:41:54] Features: 1/20 -- score: 0.7585959885386819[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:  2.9min finished

[2021-01-20 12:44:48] Features: 2/20 -- score: 0.8413323782234957[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  3.3min finished

[2021-01-20 12:48:06] Features: 3/20 -- score: 0.873567335243553[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.7min finished

[2021-01-20 12:51:50] Features: 4/20 -- score: 0.9115329512893983[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:  3.6min finished

[2021-01-20 12:55:27] Features: 5/20 -- score: 0.9262177650429799[Parallel(n_

iteracion: 3
feature: 20


[Parallel(n_jobs=-1)]: Done  23 out of  23 | elapsed:   52.6s finished

[2021-01-20 13:28:39] Features: 1/20 -- score: 0.8016469745793054[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:  2.6min finished

[2021-01-20 13:31:15] Features: 2/20 -- score: 0.8281417830290011[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  3.1min finished

[2021-01-20 13:34:19] Features: 3/20 -- score: 0.8718224131757967[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  3.0min finished

[2021-01-20 13:37:20] Features: 4/20 -- score: 0.8936627282491945[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:  3.1min finished

[2021-01-20 13:40:25] Features: 5/20 -- score: 0.9087003222341569[Parallel(n

In [ ]:
df_types = pd.DataFrame({
    'Tecnica' : pd.Series(['MLP sin Selección','MLP + SFS','MLP + SFS']),
    '# de características seleccionadas' : pd.Series([23,15,20]),
   })
df_types["Eficiencia(%)"] = eficiencia_
df_types["Error de validación"] = errorVal_
df_types["IC(std)"] = ic_
df_types["Tiempo de ejecución"] = tEjec_

df_types.set_index(['Tecnica','# de características seleccionadas'], inplace=True)
df_types.to_csv('car_select_MLP.csv')
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

Eficiencia(%)  \
Tecnica           # de características seleccionadas                      
MLP sin Selección 23                                  93.11180596916154   
MLP + SFS         15                                  95.41711421410669   
                  20                                  96.16899391335481   

                                                      Error de validación  \
Tecnica           # de características seleccionadas                        
MLP sin Selección 23                                            0.0688819   
MLP + SFS         15                                  0.03366351818966165   
                  20                                  0.03232106822520894   

                                                                   IC(std)  \
Tecnica           # de características seleccionadas                         
MLP sin Selección 23                                  0.027041804478908832   
MLP + SFS         15                                  0.010061992419702763   
                  20                                  0.006799188005530789   

                                                       Tiempo de ejecución  
Tecnica           # de características seleccionadas                        
MLP sin Selección 23                                    62.4294958114624 s  
MLP + SFS         15                                  10193.092422962189 s  
                  20                                  11881.598116636276 s

In [ ]:
model = OneVsRestClassifier(MLPClassifier(hidden_layer_sizes = (20, 20), activation = 'relu', max_iter=500), n_jobs=-1)

In [ ]:
_, _, _, _,car = trainSFS(20, model)

iteracion: 0
feature: 20


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   21.9s finished

[2021-01-18 23:50:04] Features: 1/20 -- score: 0.7800859598853869[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   45.4s finished

[2021-01-18 23:50:49] Features: 2/20 -- score: 0.8631805157593123[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   56.1s finished

[2021-01-18 23:51:45] Features: 3/20 -- score: 0.8993553008595988[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:  1.1min finished

[2021-01-18 23:52:51] Features: 4/20 -- score: 0.9151146131805158[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.1min finished



iteracion: 1
feature: 20


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   21.8s finished

[2021-01-19 00:03:48] Features: 1/20 -- score: 0.7743553008595988[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   53.3s finished

[2021-01-19 00:04:41] Features: 2/20 -- score: 0.8398997134670487[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.1min finished

[2021-01-19 00:05:46] Features: 3/20 -- score: 0.8796561604584527[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:  1.2min finished

[2021-01-19 00:06:58] Features: 4/20 -- score: 0.8972063037249284[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.1min finished

[2021-01-19 00:08:05] Features: 5/20 -- score: 0.9172636103151862[Parallel(n

iteracion: 2
feature: 20


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   20.7s finished

[2021-01-19 00:18:51] Features: 1/20 -- score: 0.7611031518624641[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:   48.5s finished

[2021-01-19 00:19:40] Features: 2/20 -- score: 0.839541547277937[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.0min finished

[2021-01-19 00:20:41] Features: 3/20 -- score: 0.8857449856733525[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:  1.1min finished

[2021-01-19 00:21:46] Features: 4/20 -- score: 0.9079512893982808[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.1min finished

[2021-01-19 00:22:51] Features: 5/20 -- score: 0.9294412607449857[Parallel(n_

iteracion: 3
feature: 20


[Parallel(n_jobs=-1)]: Done  22 out of  22 | elapsed:   35.5s finished

[2021-01-19 00:35:09] Features: 1/20 -- score: 0.8016469745793054[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  1.1min finished

[2021-01-19 00:36:17] Features: 2/20 -- score: 0.8292158968850698[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.5min finished

[2021-01-19 00:37:50] Features: 3/20 -- score: 0.8696741854636592[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:  1.6min finished

[2021-01-19 00:39:24] Features: 4/20 -- score: 0.895452918009309[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  1.8min finished

[2021-01-19 00:41:14] Features: 5/20 -- score: 0.9126387397064089[Parallel(n_

In [ ]:
car

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 21)

In [ ]:
caracteristicas

[(1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 14, 16, 17, 19, 22),
 (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 21, 22)]